In [2]:
import networkx as nx
import pandas as pd
import sys
import numpy as np

In [3]:
PATH = "../data/"

FILE_NETWORK_RETWEET_WEIGHT_EXAMPLE = "higgs-retweet_network_example.edgelist"
FILE_NETWORK_RETWEET_WEIGHT = "higgs-retweet_network.edgelist"

In [4]:
data = pd.read_csv(FILE_NETWORK_RETWEET_WEIGHT_EXAMPLE, sep=' ', names = ['source', 'target','weight'])

data

,source,target,weight
0,10,5,2
1,10,7,3
2,7,5,2
3,2,5,3
4,3,2,1
5,25,7,1
6,25,30,1
7,30,5,2


In [5]:
G1 = nx.DiGraph()
for idx,row in data.iterrows():
    G1.add_edge(row['target'], row['source'], weight= row['weight'])
    
G2 = nx.DiGraph()
for idx,row in data.iterrows():
    G2.add_edge(row['source'], row['target'], weight= row['weight'])    

In [6]:
nodes = []
for node in G1.nodes_iter():
    nodes.append(node)
    
nodes = list(set(nodes))

for n in nodes:
    
    if(G1.in_degree(n) == 0):
        plurality = sys.maxsize
    else:
        sum_weight = 0
        
        for g2_node_source in G2.neighbors(n):
            sum_weight = sum_weight + G1.get_edge_data(g2_node_source,n)['weight']
            
        plurality = int(sum_weight/2) + 1
        
    G1.node[n]['plurality'] = plurality
    
    #print('Node: {0} \t in_degree: {1} \t plurality: {2}'.format(n, G1.in_degree(n), G1.node[n]['plurality']))

In [7]:
node_to_pandas = []
lineal_threshold_to_pandas = []


In [12]:
ALERNATIVE_BOOSTRAP_NEIGHBORS_ELECTION = True

for idx, n in enumerate(nodes):
    
    print('Node: {0}'.format(n))
    
    #add to list (pandas)
    node_to_pandas.append(n)
    
    first_step_per_node = True
    
    nodes_to_add_group = []
    
    neighbors = []
    
    if(ALERNATIVE_BOOSTRAP_NEIGHBORS_ELECTION):
        neighbors.extend(G1.neighbors(n))
    else:
        neighbors.extend(G1.neighbors(n))
        neighbors.extend(G2.neighbors(n))
    
    group = []
    
    group.append(n)
    group.extend(neighbors)
    
    while( first_step_per_node or (len(nodes_to_add_group) >= 1) ):
        
        first_step_per_node = False
        
        neighbors.extend(nodes_to_add_group)
        group.extend(nodes_to_add_group)
        
        nodes_to_add_group = []
    
        #print('\t neighbors: {1}'.format(n, neighbors))

        dispersion = []

        vei = []
        for v in neighbors:
            vei.extend(G1.neighbors(v))

        dispersion = list(set(vei) - set(group))

        #print('\t dispersion {0} '.format(dispersion))

        for n_sub_level in dispersion:
            plurality = G1.node[n_sub_level]['plurality']
            #print('\t \t Reach node {0} | plurality {1}'.format(n_sub_level,plurality))

            group_influce = 0
            for node_group in group:
                if(G1.get_edge_data(node_group,n_sub_level)):
                    group_influce = group_influce + G1.get_edge_data(node_group,n_sub_level)['weight']

            #print('\t \t group {0} | influce {1}'.format(group, group_influce))

            if(group_influce >= plurality):
                nodes_to_add_group.append(n_sub_level)

        #print('\t \t \t new group {0} '.format(nodes_to_add_group))

        #print()
        

    lineal_threshold_to_pandas.append(len(neighbors) + 1)
    #print("--------------------------------------------------------------------")

Node: 2
G1.neighbors(n): [3]
G2.neighbors(n): [5]
Node: 3
G1.neighbors(n): []
G2.neighbors(n): [2]
Node: 5
G1.neighbors(n): [10, 2, 30, 7]
G2.neighbors(n): []
Node: 7
G1.neighbors(n): [25, 10]
G2.neighbors(n): [5]
Node: 10
G1.neighbors(n): []
G2.neighbors(n): [5, 7]
Node: 25
G1.neighbors(n): []
G2.neighbors(n): [30, 7]
Node: 30
G1.neighbors(n): [25]
G2.neighbors(n): [5]


In [9]:
df_centrality = pd.DataFrame(node_to_pandas, columns=["node"])
df_centrality["lineal_threshold"] = lineal_threshold_to_pandas

In [10]:
df_centrality

,node,lineal_threshold
0,2,7
1,3,2
2,5,7
3,7,7
4,10,7
5,25,4
6,30,7


In [ ]:
df_centrality.to_csv('lineal_threshold_centrality.csv', index = False )